In [ ]:
# math
import pandas as pd

# other
import copy

from experiments_2022 import IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
)
from experiments_2022.datasets import (
    load_zones,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022,
    NO_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    MULTI_PLOT_TXT_SIZE,
    MULTI_PLOT_TITLE_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
)

In [ ]:
EXCLUDED_ZONES = {}
for project in PROJECTS_2022:
    ezs = pd.read_csv(f"./dicts_dfs/excluded/{project}.csv").set_index("Unnamed: 0")
    ezs.index.name = None
    EXCLUDED_ZONES[project] = ezs

# Building level, command

In [ ]:
these_projects = copy.deepcopy(PROJECTS_2022)
these_projects.remove("LAB-3")

In [ ]:
all_results = {}
results_total_num = pd.DataFrame(
    0,
    index=["Schedule 74°F", "Schedule 76°F", "Schedule 78°F"],
    columns=[
        "Correct CSP Command (Included)",
        "Incorrect CSP Command (Included)",
        "Correct CSP Command (Excluded)",
        "Incorrect CSP Command (Excluded)",
    ],
)
results_total_denom = pd.DataFrame(
    0,
    index=["Schedule 74°F", "Schedule 76°F", "Schedule 78°F"],
    columns=[
        "Correct CSP Command (Included)",
        "Incorrect CSP Command (Included)",
        "Correct CSP Command (Excluded)",
        "Incorrect CSP Command (Excluded)",
    ],
)

for project in these_projects:
    results = pd.DataFrame(
        index=["Schedule 74°F", "Schedule 76°F", "Schedule 78°F"],
        columns=[
            "Correct CSP Command (Included)",
            "Incorrect CSP Command (Included)",
            "Correct CSP Command (Excluded)",
            "Incorrect CSP Command (Excluded)",
        ],
    )
    if project == "OFF-6":
        command = load_zones("2022", "OFF-6", "zone-zonesp") + 2
    else:
        command = load_zones("2022", project, "zone-deadband_top")

    command = cleaning.clean_df(
        df=command,
        this_var="zone-deadband_top",
        start_date=SUMMER_START_2022,
        end_date=SUMMER_END_2022,
    )

    command = command.round(0)

    for sp in [74, 76, 78]:
        this_command = cleaning.clean_df(
            df=command,
            hourly_filter=cleaning.get_experiment_hourly_filter(
                [project], "2022", [f"CSP = {sp}F"], no_weekends=NO_WEEKENDS[project]
            )[project],
            only_business_hours=True,
            no_weekends=NO_WEEKENDS[project],
            SI_units=False,
        )

        included_zones = list(this_command.columns)
        excluded_zones = list(
            (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna().index
        )
        excluded_zones = list(set(included_zones).intersection(set(excluded_zones)))
        included_zones = list(set(included_zones) - set(excluded_zones))
        this_command_included = this_command[included_zones]
        this_command_excluded = this_command[excluded_zones]

        # included
        correct = (this_command_included == sp).sum().sum() / (
            len(this_command.index) * len(this_command.columns)
        )
        incorrect = (this_command_included != sp).sum().sum() / (
            len(this_command.index) * len(this_command.columns)
        )
        results.loc[f"Schedule {sp}°F", "Correct CSP Command (Included)"] = correct
        results.loc[f"Schedule {sp}°F", "Incorrect CSP Command (Included)"] = incorrect

        # excluded
        correct = (this_command_excluded == sp).sum().sum() / (
            len(this_command.index) * len(this_command.columns)
        )
        incorrect = (this_command_excluded != sp).sum().sum() / (
            len(this_command.index) * len(this_command.columns)
        )
        results.loc[f"Schedule {sp}°F", "Correct CSP Command (Excluded)"] = correct
        results.loc[f"Schedule {sp}°F", "Incorrect CSP Command (Excluded)"] = incorrect

        # total
        results_total_num.loc[f"Schedule {sp}°F", "Correct CSP Command (Included)"] += (
            (this_command_included == sp).sum().sum()
        )
        results_total_num.loc[
            f"Schedule {sp}°F", "Incorrect CSP Command (Included)"
        ] += ((this_command_included != sp).sum().sum())
        results_total_num.loc[f"Schedule {sp}°F", "Correct CSP Command (Excluded)"] += (
            (this_command_excluded == sp).sum().sum()
        )
        results_total_num.loc[
            f"Schedule {sp}°F", "Incorrect CSP Command (Excluded)"
        ] += ((this_command_excluded != sp).sum().sum())
        results_total_denom.loc[
            f"Schedule {sp}°F", "Correct CSP Command (Included)"
        ] += len(this_command.index) * len(this_command.columns)
        results_total_denom.loc[
            f"Schedule {sp}°F", "Incorrect CSP Command (Included)"
        ] += len(this_command.index) * len(this_command.columns)
        results_total_denom.loc[
            f"Schedule {sp}°F", "Correct CSP Command (Excluded)"
        ] += len(this_command.index) * len(this_command.columns)
        results_total_denom.loc[
            f"Schedule {sp}°F", "Incorrect CSP Command (Excluded)"
        ] += len(this_command.index) * len(this_command.columns)

    all_results[project] = results
    print(project)

results_total = results_total_num / results_total_denom
all_results["TOTAL"] = results_total

In [ ]:
fig = viz.make_bar_plot(
    y_data=all_results,
    bar_legend={
        "name": {
            "Correct CSP Command (Included)": "Correct CSP Command",
            "Incorrect CSP Command (Included)": "Incorrect CSP Command",
            "Correct CSP Command (Excluded)": "Correct CSP Command (Excluded)",
            "Incorrect CSP Command (Excluded)": "Incorrect CSP Command (Excluded)",
        },
        "color": {
            "Correct CSP Command (Included)": "Green",
            "Incorrect CSP Command (Included)": "Red",
            "Correct CSP Command (Excluded)": "Green",
            "Incorrect CSP Command (Excluded)": "Red",
        },
        "opacity": {
            "Correct CSP Command (Included)": 1,
            "Incorrect CSP Command (Included)": 1,
            "Correct CSP Command (Excluded)": 0.7,
            "Incorrect CSP Command (Excluded)": 0.7,
        },
        "pattern": {
            "Correct CSP Command (Included)": "",
            "Incorrect CSP Command (Included)": "",
            "Correct CSP Command (Excluded)": "/",
            "Incorrect CSP Command (Excluded)": "/",
        },
    },
    dont_add_to_legend=[
        "Correct CSP Command (Excluded)",
        "Incorrect CSP Command (Excluded)",
    ],
    pattern_legend={"Excluded": ("/", 0.7)},
    y_axis_title="Fraction of Zone Hours<br>[Unitless]",
    vertical_spacing=0.075,
    y_range=[0, 1],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.025,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureH1.png")

# Building level, command

In [ ]:
these_projects = copy.deepcopy(PROJECTS_2022)
these_projects.remove("LAB-3")

In [ ]:
command = pull_from_dataset("2022", these_projects, "zone-deadband_top")

In [ ]:
command_74 = cleaning.clean_dfs(
    dfs=command,
    this_var="zone-deadband_top",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 74F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)

command_76 = cleaning.clean_dfs(
    dfs=command,
    this_var="zone-deadband_top",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 76F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)
command_78 = cleaning.clean_dfs(
    dfs=command,
    this_var="zone-deadband_top",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 78F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)

In [ ]:
command_74_mean = base.run_passive_test_on_dfs(
    dfs=command_74, this_test="Mean", col_name="CSP = 74°F"
)
command_76_mean = base.run_passive_test_on_dfs(
    dfs=command_76, this_test="Mean", col_name="CSP = 76°F"
)
command_78_mean = base.run_passive_test_on_dfs(
    dfs=command_78, this_test="Mean", col_name="CSP = 78°F"
)

command_74_std = base.run_passive_test_on_dfs(
    dfs=command_74, this_test="Std", col_name="CSP = 74°F"
)
command_76_std = base.run_passive_test_on_dfs(
    dfs=command_76, this_test="Std", col_name="CSP = 76°F"
)
command_78_std = base.run_passive_test_on_dfs(
    dfs=command_78, this_test="Std", col_name="CSP = 78°F"
)

In [ ]:
command_mean = base.combine_dicts([command_74_mean, command_76_mean, command_78_mean])
command_std = base.combine_dicts([command_74_std, command_76_std, command_78_std])

In [ ]:
for project in these_projects:
    these_zones = list(command_mean[project].index)
    these_ezs = list(
        (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna().index
    )
    these_zones = list(set(these_zones) - set(these_ezs))
    command_mean[project] = command_mean[project].loc[these_zones, :]
    command_std[project] = command_std[project].loc[these_zones, :]

In [ ]:
opacity_data = {}
for project in these_projects:
    opacity_data[project] = pd.Series(
        0.8, index=list(command_mean[project].index)
    ).to_frame()

In [ ]:
fig = viz.make_dot_plot(
    y_data=command_mean,
    y_error_up_data=command_std,
    y_error_down_data=command_std,
    normalize_x=True,
    sort_by="CSP = 74°F",
    y_axis_title="CSP Command [°F]",
    x_axis_title="Fraction of Included Zones [Unitless]",
    horizontal_spacing=0.075,
    vertical_spacing=0.125,
    color_legend={
        "color": {
            "CSP = 74°F": "RoyalBlue",
            "CSP = 76°F": "DarkOrange",
            "CSP = 78°F": "Firebrick",
        }
    },
    opacity_data=opacity_data,
    y_range=[70, 85],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureH2.png")

# Building level, effective

In [ ]:
effective = pull_from_dataset("2022", PROJECTS_2022, "zone-coolsp")

In [ ]:
effective_74 = cleaning.clean_dfs(
    dfs=effective,
    this_var="zone-coolsp",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 74F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)

effective_76 = cleaning.clean_dfs(
    dfs=effective,
    this_var="zone-coolsp",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 76F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)
effective_78 = cleaning.clean_dfs(
    dfs=effective,
    this_var="zone-coolsp",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_2022,
        experiment_year="2022",
        filter_columns=["CSP = 78F"],
        no_weekends=NO_WEEKENDS,
        use_raw=False,
    ),
)

In [ ]:
effective_74_mean = base.run_passive_test_on_dfs(
    dfs=effective_74, this_test="Mean", col_name="CSP = 74°F"
)
effective_76_mean = base.run_passive_test_on_dfs(
    dfs=effective_76, this_test="Mean", col_name="CSP = 76°F"
)
effective_78_mean = base.run_passive_test_on_dfs(
    dfs=effective_78, this_test="Mean", col_name="CSP = 78°F"
)

effective_74_std = base.run_passive_test_on_dfs(
    dfs=effective_74, this_test="Std", col_name="CSP = 74°F"
)
effective_76_std = base.run_passive_test_on_dfs(
    dfs=effective_76, this_test="Std", col_name="CSP = 76°F"
)
effective_78_std = base.run_passive_test_on_dfs(
    dfs=effective_78, this_test="Std", col_name="CSP = 78°F"
)

In [ ]:
effective_mean = base.combine_dicts(
    [effective_74_mean, effective_76_mean, effective_78_mean]
)
effective_std = base.combine_dicts(
    [effective_74_std, effective_76_std, effective_78_std]
)

In [ ]:
for project in PROJECTS_2022:
    these_zones = list(effective_mean[project].index)
    these_ezs = list(
        (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna().index
    )
    these_zones = list(set(these_zones) - set(these_ezs))
    effective_mean[project] = effective_mean[project].loc[these_zones, :]
    effective_std[project] = effective_std[project].loc[these_zones, :]

In [ ]:
opacity_data = {}
for project in PROJECTS_2022:
    opacity_data[project] = pd.Series(
        0.8, index=list(effective_mean[project].index)
    ).to_frame()

In [ ]:
fig = viz.make_dot_plot(
    y_data=effective_mean,
    y_error_up_data=effective_std,
    y_error_down_data=effective_std,
    normalize_x=True,
    sort_by="CSP = 74°F",
    y_axis_title="Effective CSP [°F]",
    x_axis_title="Fraction of Included Zones [Unitless]",
    horizontal_spacing=0.075,
    vertical_spacing=0.1,
    color_legend={
        "color": {
            "CSP = 74°F": "RoyalBlue",
            "CSP = 76°F": "DarkOrange",
            "CSP = 78°F": "Firebrick",
        }
    },
    opacity_data=opacity_data,
    y_range=[70, 85],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureH3.png")

# Zone level, effective

In [ ]:
effective = pull_from_dataset("2022", PROJECTS_2022, "zone-coolsp")

In [ ]:
effective = cleaning.clean_dfs(
    dfs=effective,
    this_var="zone-coolsp",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
)

In [ ]:
zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=effective[project]
    )

In [ ]:
df_filter_74 = cleaning.create_sp_filter(
    zonal_schedules, sps=[74], reverse_filter=False
)
effective_74 = cleaning.clean_dfs(
    dfs=effective, only_business_hours=False, no_weekends=False, df_filter=df_filter_74
)

In [ ]:
df_filter_76 = cleaning.create_sp_filter(
    zonal_schedules, sps=[76], reverse_filter=False
)
effective_76 = cleaning.clean_dfs(
    dfs=effective, only_business_hours=False, no_weekends=False, df_filter=df_filter_76
)

In [ ]:
df_filter_78 = cleaning.create_sp_filter(
    zonal_schedules, sps=[78], reverse_filter=False
)
effective_78 = cleaning.clean_dfs(
    dfs=effective, only_business_hours=False, no_weekends=False, df_filter=df_filter_78
)

In [ ]:
effective_74_mean = base.run_passive_test_on_dfs(
    dfs=effective_74, this_test="Mean", col_name="CSP = 74°F"
)
effective_76_mean = base.run_passive_test_on_dfs(
    dfs=effective_76, this_test="Mean", col_name="CSP = 76°F"
)
effective_78_mean = base.run_passive_test_on_dfs(
    dfs=effective_78, this_test="Mean", col_name="CSP = 78°F"
)

effective_74_std = base.run_passive_test_on_dfs(
    dfs=effective_74, this_test="Std", col_name="CSP = 74°F"
)
effective_76_std = base.run_passive_test_on_dfs(
    dfs=effective_76, this_test="Std", col_name="CSP = 76°F"
)
effective_78_std = base.run_passive_test_on_dfs(
    dfs=effective_78, this_test="Std", col_name="CSP = 78°F"
)

In [ ]:
effective_mean = base.combine_dicts(
    [effective_74_mean, effective_76_mean, effective_78_mean]
)
effective_std = base.combine_dicts(
    [effective_74_std, effective_76_std, effective_78_std]
)

In [ ]:
for project in PROJECTS_2022:
    these_zones = list(effective_mean[project].index)
    these_ezs = list(
        (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna().index
    )
    these_zones = list(set(these_zones) - set(these_ezs))
    effective_mean[project] = effective_mean[project].loc[these_zones, :]
    effective_std[project] = effective_std[project].loc[these_zones, :]

In [ ]:
opacity_data = {}
for project in these_projects:
    opacity_data[project] = pd.Series(
        0.8, index=list(effective_mean[project].index)
    ).to_frame()

In [ ]:
fig = viz.make_dot_plot(
    y_data=effective_mean,
    y_error_up_data=effective_std,
    y_error_down_data=effective_std,
    normalize_x=True,
    sort_by="CSP = 74°F",
    y_axis_title="Effective CSP [°F]",
    x_axis_title="Fraction of Included Zones [Unitless]",
    horizontal_spacing=0.075,
    vertical_spacing=0.1,
    color_legend={
        "color": {
            "CSP = 74°F": "RoyalBlue",
            "CSP = 76°F": "DarkOrange",
            "CSP = 78°F": "Firebrick",
        }
    },
    opacity_data=opacity_data,
    y_range=[70, 85],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureH5.png")

## Zone level, command

In [ ]:
these_projects = copy.deepcopy(PROJECTS_2022)
these_projects.remove("LAB-3")

In [ ]:
command = pull_from_dataset("2022", these_projects, "zone-deadband_top")

In [ ]:
command = cleaning.clean_dfs(
    dfs=command,
    this_var="zone-deadband_top",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=False,
)

In [ ]:
zonal_schedules = {}
for project in these_projects:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=command[project]
    )

In [ ]:
df_filter_74 = cleaning.create_sp_filter(
    zonal_schedules, sps=[74], reverse_filter=False
)
command_74 = cleaning.clean_dfs(
    dfs=command, only_business_hours=False, no_weekends=False, df_filter=df_filter_74
)

In [ ]:
df_filter_76 = cleaning.create_sp_filter(
    zonal_schedules, sps=[76], reverse_filter=False
)
command_76 = cleaning.clean_dfs(
    dfs=command, only_business_hours=False, no_weekends=False, df_filter=df_filter_76
)

In [ ]:
df_filter_78 = cleaning.create_sp_filter(
    zonal_schedules, sps=[78], reverse_filter=False
)
command_78 = cleaning.clean_dfs(
    dfs=command, only_business_hours=False, no_weekends=False, df_filter=df_filter_78
)

In [ ]:
command_74_mean = base.run_passive_test_on_dfs(
    dfs=command_74, this_test="Mean", col_name="CSP = 74°F"
)
command_76_mean = base.run_passive_test_on_dfs(
    dfs=command_76, this_test="Mean", col_name="CSP = 76°F"
)
command_78_mean = base.run_passive_test_on_dfs(
    dfs=command_78, this_test="Mean", col_name="CSP = 78°F"
)

command_74_std = base.run_passive_test_on_dfs(
    dfs=command_74, this_test="Std", col_name="CSP = 74°F"
)
command_76_std = base.run_passive_test_on_dfs(
    dfs=command_76, this_test="Std", col_name="CSP = 76°F"
)
command_78_std = base.run_passive_test_on_dfs(
    dfs=command_78, this_test="Std", col_name="CSP = 78°F"
)

In [ ]:
command_mean = base.combine_dicts([command_74_mean, command_76_mean, command_78_mean])
command_std = base.combine_dicts([command_74_std, command_76_std, command_78_std])

In [ ]:
for project in these_projects:
    these_zones = list(command_mean[project].index)
    these_ezs = list(
        (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna().index
    )
    these_zones = list(set(these_zones) - set(these_ezs))
    command_mean[project] = command_mean[project].loc[these_zones, :]
    command_std[project] = command_std[project].loc[these_zones, :]

In [ ]:
opacity_data = {}
for project in these_projects:
    opacity_data[project] = pd.Series(
        0.8, index=list(command_mean[project].index)
    ).to_frame()

In [ ]:
fig = viz.make_dot_plot(
    y_data=command_mean,
    y_error_up_data=command_std,
    y_error_down_data=command_std,
    normalize_x=True,
    sort_by="CSP = 74°F",
    y_axis_title="CSP Command [°F]",
    x_axis_title="Fraction of Included Zones [Unitless]",
    horizontal_spacing=0.075,
    vertical_spacing=0.125,
    color_legend={
        "color": {
            "CSP = 74°F": "RoyalBlue",
            "CSP = 76°F": "DarkOrange",
            "CSP = 78°F": "Firebrick",
        }
    },
    opacity_data=opacity_data,
    y_range=[70, 85],
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureH4.png")